In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# General imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
import seaborn as sns
from mlxtend.plotting import plot_decision_regions
from sklearn.metrics import confusion_matrix
import cv2
import PIL

In [3]:
# Set up ImageDataGenerator
train_imagegen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                   shear_range=20,
                                   zoom_range=0.2,
                                   rotation_range=20,                           
                                   horizontal_flip=True,
                                   validation_split=0.06) # this will set aside a part of training set for validation data
test_imagegen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                   shear_range=20,
                                   zoom_range=0.2,
                                   rotation_range=20, 
                                   horizontal_flip=True)
# Bring the data in
train_generator = train_imagegen.flow_from_directory(
                                    '../input/brain-tumor-classification-mri/Training',
                                    target_size=(150,150),
                                    batch_size=20,
                                    class_mode='categorical',
                                    subset='training')

test_generator = test_imagegen.flow_from_directory(
                                    '../input/brain-tumor-classification-mri/Testing',
                                    target_size=(150,150),
                                    batch_size=20,
                                    class_mode='categorical')

val_generator = train_imagegen.flow_from_directory(
                                    '../input/brain-tumor-classification-mri/Training',
                                    target_size=(150,150),
                                    batch_size=20,
                                    class_mode='categorical',
                                    subset='validation')

In [4]:
# Visualize (code from https://github.com/austint1121/OES-PneumoniaClassification/blob/main/Final_Notebook.ipynb)
train_batch = train_generator.next()
fig, axes = plt.subplots(2, 5, figsize=(16, 8))
    
for i in range(10):
    # Load image into numpy array and re-scale
    img = np.array(train_batch[0][i] * 255, dtype='uint8')
    ax = axes[i // 5, i % 5]
    ax.imshow(img)
fig.suptitle('Training Images')
plt.tight_layout()
plt.show()

## **Getting a look at the nested numpy array for images in order to verify whether or not it uses all three color channels**


In [ ]:
train_generator.next()

In [ ]:
len(train_generator.next()[0])

In [ ]:
train_generator.next()[0][0]

In [ ]:
len(train_generator.next()[0][0])

In [ ]:
train_generator.next()[0][0][0]

In [ ]:
len(train_generator.next()[0][0][0])

In [ ]:
train_generator.next()[0][0][0][0]

## **Getting a look at the Distribution of Different Tumor Types**

In [5]:
train_generator.class_indices

In [ ]:
train_generator.classes

In [6]:
tumor_types = pd.DataFrame(train_generator.classes)
values = tumor_types.value_counts()
values

In [7]:
tumor_types

In [8]:
train_generator.classes


In [ ]:
tumor_dict = {0: 'Glioma',
             1: 'Meningioma',
             2: 'No tumor',
             3: 'Pituitary'}

In [ ]:
tumor_types = tumor_types[0].map(tumor_dict)
tumor_types.value_counts()

In [ ]:
plt.figure(figsize=(10,6))
sns.set(font_scale=1.4)
sns.barplot(tumor_types.index, tumor_types.values)
plt.ylabel("Number of Images")
plt.title('Distribution of Brain Tumor Images');

## **Taking a Look at Different Tumor Types**

In [ ]:
# View a Glioma tumor
glioma1 = PIL.Image.open('../input/brain-tumor-classification-mri/Training/glioma_tumor/gg (1).jpg')
glioma1

In [ ]:
glioma1.mode

In [ ]:
glioma1.size

In [ ]:
# View another Glioma
PIL.Image.open('../input/brain-tumor-classification-mri/Training/glioma_tumor/gg (103).jpg')

In [ ]:
# View another Glioma
PIL.Image.open('../input/brain-tumor-classification-mri/Training/glioma_tumor/gg (10).jpg')

In [ ]:
# View another Glioma
PIL.Image.open('../input/brain-tumor-classification-mri/Training/glioma_tumor/gg (12).jpg')

In [ ]:
# View another Glioma
PIL.Image.open('../input/brain-tumor-classification-mri/Training/glioma_tumor/gg (13).jpg')

In [ ]:
# View a meningioma_tumor
PIL.Image.open('../input/brain-tumor-classification-mri/Training/meningioma_tumor/m (109).jpg')

In [ ]:
# View another meningioma_tumor
PIL.Image.open('../input/brain-tumor-classification-mri/Training/meningioma_tumor/m (117).jpg')

In [ ]:
# View another meningioma_tumor
PIL.Image.open('../input/brain-tumor-classification-mri/Training/meningioma_tumor/m (143).jpg')

In [ ]:
# View a pituitary tumor
PIL.Image.open('../input/brain-tumor-classification-mri/Training/pituitary_tumor/p (112).jpg')

In [ ]:
# View another pituitary tumor
PIL.Image.open('../input/brain-tumor-classification-mri/Training/pituitary_tumor/p (1).jpg')

In [ ]:
# View a normal image
normal1 = PIL.Image.open('../input/brain-tumor-classification-mri/Training/no_tumor/8.jpg')
normal1

In [ ]:
print(normal1.size)

In [ ]:
# View another normal image
normal2 = PIL.Image.open('../input/brain-tumor-classification-mri/Training/no_tumor/image (20).jpg')
normal2

In [ ]:
normal2.size

In [ ]:
# View another normal image
normal3 = PIL.Image.open('../input/brain-tumor-classification-mri/Training/no_tumor/image (61).jpg')
normal3

In [ ]:
normal3.size